# 1. Uninformed Search

This section covers several search strategies that come under the heading of **uninformed search** (also called **blind search**). The term means that the strategies have no additional information about states beyond that provided in the problem definition. All they can do is generate successors and distinguish a goal state from a non-goal state. All search strategies are distinguished by the order in which nodes are expanded. Strategies that know whether one non-goal state is “more promising” than another are called **informed search** or **heuristic search** strategies; they are covered in Section 2.

## Breadth first search (BFS)

**Breadth-first search** is a simple strategy in which the root node is expanded first, then all the successors of the root node are expanded next, then their successors, and so on. In general, all the nodes are expanded at a given depth in the search tree before any nodes at the next level are expanded.

![BFS](images/bfs.png "BFS")

In [1]:
def BFS(graph, start, goal):
    queue = []
    visited = []
    queue.append(start)
    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.append(node)
            if node == goal:
                return visited
            for neighbor in graph[node]:
                if neighbor not in visited:
                    queue.append(neighbor)
    return visited

In [2]:
graph = {'A': ['B', 'C'],
         'B': ['A', 'D', 'E'],
         'C': ['A', 'F', 'G'],
         'D': ['B'],
         'E': ['B'],
         'F': ['C'],
         'G': ['C']}

ans = BFS(graph, 'A', 'G')
print(ans)

['A', 'B', 'C', 'D', 'E', 'F', 'G']


## Depth first search (DFS)

**Depth-first search** always expands the *deepest* node in the current frontier of the search tree. The progress of the search is illustrated in Figure 3.16. The search proceeds immediately to the deepest level of the search tree, where the nodes have no successors. As those nodes are expanded, they are dropped from the frontier, so then the search “backs up” to the next deepest node that still has unexplored successors.

![DFS](images/dfs.png "DFS")

In [3]:
def DFS(graph, start, goal):
    stack = []
    visited = []
    stack.append(start)
    while stack:
        node = stack.pop()
        if node not in visited:
            visited.append(node)
            if node == goal:
                return visited
            for neighbor in graph[node]:
                stack.append(neighbor)
    return visited

In [4]:
graph = {'A': ['C', 'B'],
         'B': ['A', 'E', 'D'],
         'C': ['A', 'G', 'F'],
         'D': ['B', 'I', 'H'],
         'E': ['B', 'K', 'J'],
         'F': ['C', 'M', 'L'],
         'G': ['C', 'O', 'N'],
         'H': ['D'],
         'I': ['D'],
         'J': ['E'],
         'K': ['E'],
         'L': ['F'],
         'M': ['F'],
         'N': ['G'],
         'O': ['G']}

ans = DFS(graph, 'A', 'M')
print(ans)

['A', 'B', 'D', 'H', 'I', 'E', 'J', 'K', 'C', 'F', 'L', 'M']


## Uniform cost search (UCS)

When all step costs are equal, breadth-first search is optimal because it always expands the *shallowest* unexpanded node. By a simple extension, we can find an algorithm that is optimal with any step-cost function. Instead of expanding the shallowest node, **uniform-cost search** expands the node $n$ with the *lowest path cost* $g(n).$ This is done by storing the frontier as a priority queue ordered by $g$

In [17]:
import queue as Q

def UCS(graph, start, end):
    if start not in graph:
        raise TypeError(str(start) + ' not found in graph !')
        return
    if end not in graph:
        raise TypeError(str(end) + ' not found in graph !')
        return
    
    queue = Q.PriorityQueue()
    queue.put((0, [start]))
    
    while not queue.empty():
        node = queue.get()
        current = node[1][len(node[1]) - 1]
        
        if end in node[1]:
            return node[1], node[0]
            break
        
        cost = node[0]
        for neighbor in graph[current]:
            temp = node[1][:]
            temp.append(neighbor)
            queue.put((cost + graph[current][neighbor], temp))

In [21]:
graph = {
    'Bucharest': {'Pitesti': 101, 'Giurgiu': 90, 'Fagaras': 211}, 
    'RimnicuVilcea': {'Pitesti': 97, 'Sibiu': 80, 'Craiova': 146}, 
    'Craiova': {'Pitesti': 138, 'Dobreta': 120, 'RimnicuVilcea': 146}, 
    'Oradea': {'Zerind': 71, 'Sibiu': 151}, 
    'Giurgiu': {'Bucharest': 90}, 
    'Dobreta': {'Mehadia': 75, 'Craiova': 120}, 
    'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118}, 
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70}, 
    'Sibiu': {'RimnicuVilcea': 80, 'Arad': 140, 'Fagaras': 99, 'Oradea': 151}, 
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211}, 
    'Zerind': {'Arad': 75, 'Oradea': 71}, 
    'Mehadia': {'Dobreta': 75, 'Lugoj': 70}, 
    'Pitesti': {'RimnicuVilcea': 97, 'Craiova': 138, 'Bucharest': 101}, 
    'Timisoara': {'Arad': 118, 'Lugoj': 111}}

path, cost = UCS(graph, 'Arad', 'Bucharest')
print("Path found: " + str(path) + ", Cost = " + str(cost))

Path found: ['Arad', 'Sibiu', 'RimnicuVilcea', 'Pitesti', 'Bucharest'], Cost = 418
